In [146]:
%matplotlib inline
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import scipy.stats as stats
import sklearn.linear_model as linear_model
import seaborn as sns
import xgboost as xgb
from sklearn.model_selection import KFold
from IPython.display import HTML, display
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
 

pd.options.display.max_rows = 1000
pd.options.display.max_columns = 20

df = pd.read_csv('train.csv')
# test = pd.read_csv('test.csv')

quantitative = [f for f in df.columns if df.dtypes[f] != 'object']
quantitative.remove('SalePrice')
quantitative.remove('Id')
qualitative = [f for f in df.columns if df.dtypes[f] == 'object']

In [147]:
df.drop('Id', axis=1, inplace=True)

In [143]:
#add a row of zeros to test set. Problem with read 1 to few rows
test = pd.read_csv('test_data.csv')
test.loc[1460] = np.zeros(shape=(len(test.columns)))

In [181]:
#spli dataset into numerical and categorical features
test_quantitative = [f for f in test.columns if test.dtypes[f] != 'object']
test_qualitative = [f for f in test.columns if test.dtypes[f] == 'object']

In [148]:
df.shape, test.shape

((1460, 80), (1460, 79))

In [9]:
def get_scores(prediction,labels):
    print('R2 score: {}'.format(R2_score(predictions, labels)))
    print('RMSE: {}'.format(npsqrt(mean_squared_error(predictions, labels))))

#Show scores for train and validation set
def train_test(estimator, x_trn,x_test, y_trn, y_test):
    prediction_train = estmator.predict(x_trn)
    print(estimator)                                #Printing estimator 
    get_scores(prediction_train, y_trn)             #Printing train scores
    prediction_test = estimator.predict(x_test)
    print('Test')                                   #Printing Test scores
    get_scores(predictions_test,y_test)

# Dealing with null values

meikegw 'Filling Up Missing Values'


In [149]:

# Look at Categorical values
def cat_exploration(column):
    return df[column].value_counts()


# Impute Categorical variables
def cat_imputer(column, value):
    df.loc[df[column].isnull(), column] = value

df['LotFrontage'].corr(df['LotArea'])# check correlation of Lot area and Lot frontage
sqrtarea = np.sqrt(df['LotArea'])
df['SqrtLotArea'] = sqrtarea
df['LotFrontage'].corr(df['SqrtLotArea'])


import seaborn as sns
# sns.pairplot(df[['LotFrontage','SqrtLotArea']].dropna())
cond = df['LotFrontage'].isnull()  # define all instances of condos
df.LotFrontage[cond] = df.SqrtLotArea[cond] # make nulls equal to square root of lot area
del df['SqrtLotArea']

# Alley
cat_exploration('Alley')
cat_imputer('Alley', 'None')

# Masonry Veneer Type
df[['MasVnrType', 'MasVnrArea']][df['MasVnrType'].isnull()==True]
cat_exploration('MasVnrType')
cat_imputer('MasVnrType', 'None')
cat_imputer('MasVnrArea', 0.0)

# Basement
basement_col = ['BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'BsmtFinSF1', 'BsmtFinSF2']
df[basement_col][df['BsmtQual'].isnull() == True]
for col in basement_col:
    if 'FinSF' not in col:
        cat_imputer(col, 'None')#categorical cariables are null where numerical are 0. Therefore there is no basement and NaN is None
    else:
        cat_imputer(col, 0.0)

# Electrical
cat_exploration('Electrical')
cat_imputer('Electrical', 'SBrkr')        #Impute most frequent value


# Fireplace

df['Fireplaces'][df['FireplaceQu'].isnull() == True].describe()
cat_exploration('FireplaceQu')
cat_imputer('FireplaceQu', 'None')
# pd.crosstab(df.Fireplaces, df.FireplaceQu)

# Garage
garage_col = ['GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageQual', 'GarageCond', 'GarageCars', 'GarageArea']
df[garage_col][df['GarageType'].isnull() == True]

#Some residences have no garage
for col in garage_col:
    if df[col].dtype == np.object:
        cat_imputer(col, 'None')
    else:
        cat_imputer(col,0)

# Pool
df['PoolArea'][df['PoolQC'].isnull() == True].describe()
cat_imputer('PoolQC', 'None')

# Fence
cat_imputer('Fence', 'None')

# Miscellaneous Features
cat_imputer('MiscFeature','None')



C:\Users\bobbyd\Anaconda2\envs\neural_net\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [182]:
# Look at Categorical test values
def cat_exploration(column):
    return test[column].value_counts()


# Impute Categorical variables
def cat_imputer(column, value):
    test.loc[test[column].isnull(), column] = value
    
# MSZoning
cat_imputer('MSZoning', 'None')

# Utilities
cat_imputer('Utilities', 'AllPub')

test['LotFrontage'].corr(test['LotArea'])# check correlation of Lot area and Lot frontage
sqrtarea = np.sqrt(test['LotArea'])
test['SqrtLotArea'] = sqrtarea
test['LotFrontage'].corr(test['SqrtLotArea'])


import seaborn as sns
# sns.pairplot(df[['LotFrontage','SqrtLotArea']].dropna())
cond = test['LotFrontage'].isnull()  # define all instances of condos
test.LotFrontage[cond] = test.SqrtLotArea[cond] # make nulls equal to square root of lot area
del test['SqrtLotArea']

# Alley
cat_exploration('Alley')
cat_imputer('Alley', 'None')

# Masonry Veneer Type
test[['MasVnrType', 'MasVnrArea']][test['MasVnrType'].isnull()==True]
cat_exploration('MasVnrType')
cat_imputer('MasVnrType', 'None')
cat_imputer('MasVnrArea', 0.0)

# Basement
basement_col = ['BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'BsmtFinSF1', 'BsmtFinSF2']
test[basement_col][test['BsmtQual'].isnull() == True]
for col in basement_col:
    if 'FinSF' not in col:
        cat_imputer(col, 'None')#categorical cariables are null where numerical are 0. Therefore there is no basement and NaN is None
    else:
        cat_imputer(col, 0.0)

cat_imputer('TotalBsmtSF', 0.0)
cat_imputer('BsmtFullBath', 0.0)
cat_imputer('BsmtHalfBath', 0.0)
cat_imputer('BsmtUnfSF', 0.0)

#Exterior
cat_imputer('Exterior1st', 'VinylSd')
cat_imputer('Exterior2nd', 'VinylSd')

    

# Electrical
cat_exploration('Electrical')
cat_imputer('Electrical', 'SBrkr')        #Impute most frequent value

# Kitchen
cat_imputer('KitchenQual', 'TA')

# Functional
cat_imputer('Functional', 'Typ')


# Fireplace
test['Fireplaces'][test['FireplaceQu'].isnull() == True].describe()
cat_exploration('FireplaceQu')
cat_imputer('FireplaceQu', 'None')
# pd.crosstab(df.Fireplaces, df.FireplaceQu)

# Garage
garage_col = ['GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageQual', 'GarageCond', 'GarageCars', 'GarageArea']
test[garage_col][test['GarageType'].isnull() == True]

#Some residences have no garage
for col in garage_col:
    if test[col].dtype == np.object:
        cat_imputer(col, 'None')
    else:
        cat_imputer(col,0)

# Pool
test['PoolArea'][test['PoolQC'].isnull() == True].describe()
cat_imputer('PoolQC', 'None')

# Fence
cat_imputer('Fence', 'None')

# Miscellaneous Features
cat_imputer('MiscFeature','None')

# Saletype
cat_imputer('SaleType', 'Oth')



C:\Users\bobbyd\Anaconda2\envs\neural_net\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [151]:
def encode(frame, feature):
    ordering = pd.DataFrame()
    ordering['val'] = frame[feature].unique()
    ordering.index = ordering.val
    ordering['spmean'] = frame[[feature, 'SalePrice']].groupby(feature).mean()['SalePrice']
    ordering = ordering.sort_values('spmean')
    ordering['ordering'] = range(1, ordering.shape[0]+1)
    ordering = ordering['ordering'].to_dict()
    
    for cat, o in ordering.items():
        frame.loc[frame[feature] == cat, feature+'_E'] = o
    
qual_encoded = []
for q in qualitative:  
    encode(df, q)
    qual_encoded.append(q+'_E')
print(qual_encoded)

['MSZoning_E', 'Street_E', 'Alley_E', 'LotShape_E', 'LandContour_E', 'Utilities_E', 'LotConfig_E', 'LandSlope_E', 'Neighborhood_E', 'Condition1_E', 'Condition2_E', 'BldgType_E', 'HouseStyle_E', 'RoofStyle_E', 'RoofMatl_E', 'Exterior1st_E', 'Exterior2nd_E', 'MasVnrType_E', 'ExterQual_E', 'ExterCond_E', 'Foundation_E', 'BsmtQual_E', 'BsmtCond_E', 'BsmtExposure_E', 'BsmtFinType1_E', 'BsmtFinType2_E', 'Heating_E', 'HeatingQC_E', 'CentralAir_E', 'Electrical_E', 'KitchenQual_E', 'Functional_E', 'FireplaceQu_E', 'GarageType_E', 'GarageFinish_E', 'GarageQual_E', 'GarageCond_E', 'PavedDrive_E', 'PoolQC_E', 'Fence_E', 'MiscFeature_E', 'SaleType_E', 'SaleCondition_E']


In [226]:
#map ordering from train encoding to test set
def map_encoded(col):
    ordering = pd.DataFrame()
    ordering['val'] = df[col].unique()
    ordering.index = ordering.val
    ordering['spmean'] = df[[col, 'SalePrice']].groupby(col).mean()['SalePrice']
    ordering = ordering.sort_values('spmean')
    ordering['ordering'] = range(1, ordering.shape[0]+1)
    ordering = ordering['ordering'].to_dict()
    test[col+'_E_test']=test[col].map(ordering)




# Regression

In [153]:
# transform features according to skew and add new features to dataset
def error(actual, predicted):
    actual = np.log(actual)
    predicted = np.log(predicted)
    return np.sqrt(np.sum(np.square(actual-predicted))/len(actual))

def log_transform(feature):
    df[feature] = np.log1p(df[feature].values)

def quadratic(feature):
    df[feature+'2'] = df[feature]**2
    
log_transform('GrLivArea')
log_transform('1stFlrSF')
log_transform('2ndFlrSF')
log_transform('TotalBsmtSF')
log_transform('LotArea')
log_transform('LotFrontage')
log_transform('KitchenAbvGr')
log_transform('GarageArea')

quadratic('OverallQual')
quadratic('YearBuilt')
quadratic('YearRemodAdd')
quadratic('TotalBsmtSF')
quadratic('2ndFlrSF')
quadratic('Neighborhood_E')
quadratic('RoofMatl_E')
quadratic('GrLivArea')

qdr = ['OverallQual2', 'YearBuilt2', 'YearRemodAdd2', 'TotalBsmtSF2',
        '2ndFlrSF2', 'Neighborhood_E2', 'RoofMatl_E2', 'GrLivArea2']


df['HasBasement'] = df['TotalBsmtSF'].apply(lambda x: 1 if x > 0 else 0)
df['HasGarage'] = df['GarageArea'].apply(lambda x: 1 if x > 0 else 0)
df['Has2ndFloor'] = df['2ndFlrSF'].apply(lambda x: 1 if x > 0 else 0)
df['HasMasVnr'] = df['MasVnrArea'].apply(lambda x: 1 if x > 0 else 0)
df['HasWoodDeck'] = df['WoodDeckSF'].apply(lambda x: 1 if x > 0 else 0)
df['HasPorch'] = df['OpenPorchSF'].apply(lambda x: 1 if x > 0 else 0)
df['HasPool'] = df['PoolArea'].apply(lambda x: 1 if x > 0 else 0)
df['IsNew'] = df['YearBuilt'].apply(lambda x: 1 if x > 2000 else 0)

boolean = ['HasBasement', 'HasGarage', 'Has2ndFloor', 'HasMasVnr', 'HasWoodDeck',
            'HasPorch', 'HasPool', 'IsNew']



In [184]:
for col in test_qualitative:
    map_encoded(col)

In [185]:

def error(actual, predicted):
    actual = np.log(actual)
    predicted = np.log(predicted)
    return np.sqrt(np.sum(np.square(actual-predicted))/len(actual))

def log_transform(feature):
    test[feature+'_test'] = np.log1p(test[feature].values)

def quadratic(feature):
    test[feature+'_test_2'] = test[feature]**2
    
log_transform('GrLivArea')
log_transform('1stFlrSF')
log_transform('2ndFlrSF')
log_transform('TotalBsmtSF')
log_transform('LotArea')
log_transform('LotFrontage')
log_transform('KitchenAbvGr')
log_transform('GarageArea')

quadratic('OverallQual')
quadratic('YearBuilt')
quadratic('YearRemodAdd')
quadratic('TotalBsmtSF')
quadratic('2ndFlrSF')
quadratic('Neighborhood_E_test')
quadratic('RoofMatl_E_test')
quadratic('GrLivArea')

qdr_test = ['OverallQual_test_2', 'YearBuilt_test_2', 'YearRemodAdd_test_2', 'TotalBsmtSF_test_2',
        '2ndFlrSF_test_2', 'Neighborhood_E_test_test_2', 'RoofMatl_E_test_test_2', 'GrLivArea_test_2']


test['HasBasement_test'] = test['TotalBsmtSF'].apply(lambda x: 1 if x > 0 else 0)
test['HasGarage_test'] = test['GarageArea'].apply(lambda x: 1 if x > 0 else 0)
test['Has2ndFloor_test'] = test['2ndFlrSF'].apply(lambda x: 1 if x > 0 else 0)
test['HasMasVnr_test'] = test['MasVnrArea'].apply(lambda x: 1 if x > 0 else 0)
test['HasWoodDeck_test'] = test['WoodDeckSF'].apply(lambda x: 1 if x > 0 else 0)
test['HasPorch_test'] = test['OpenPorchSF'].apply(lambda x: 1 if x > 0 else 0)
test['HasPool_test'] = test['PoolArea'].apply(lambda x: 1 if x > 0 else 0)
test['IsNew_test'] = test['YearBuilt'].apply(lambda x: 1 if x > 2000 else 0)

boolean_test = ['HasBasement_test', 'HasGarage_test', 'Has2ndFloor_test', 'HasMasVnr_test', 'HasWoodDeck_test',
            'HasPorch_test', 'HasPool_test', 'IsNew_test']


    

In [186]:
#define qual_encoded_test list
qual_encoded_test =[]
for col in test.columns:
    if '_E_' in col and '_2' not in col:
        qual_encoded_test.append(col)

In [187]:
#define list of features for train and test for model prediction
features_test = qdr_test + boolean_test + test_quantitative +qual_encoded_test
features = quantitative + qual_encoded +boolean + qdr


In [191]:
#scale feature set
X = df[features].fillna(0.).values
std = StandardScaler()
s = std.fit_transform(X)


X_test= test[features_test].fillna(0.).values


ss = StandardScaler()
ss.fit_transform(X_test)

array([[-0.77547895, -0.34932312, -1.07435888, ...,  0.18791096,
        -0.21343526,  0.18971145],
       [-0.16682961, -0.44792228, -1.21495087, ..., -2.52656519,
        -0.21343526,  0.18971145],
       [-0.77547895,  0.84564133,  0.67730698, ...,  0.18791096,
        -0.21343526,  0.18971145],
       ..., 
       [-0.77547895, -0.38220628,  0.58178527, ...,  0.18791096,
        -0.21343526, -3.22743672],
       [-0.77547895,  0.67837415,  0.39102884, ..., -5.24104135,
        -0.21343526,  0.18971145],
       [ 0.55248325,  0.71179404,  0.48635922, ...,  0.18791096,
        -0.21343526,  0.18971145]])

In [161]:
#LarsCV Lasso Linear Model
lasso = linear_model.LassoLarsCV(max_iter=1000)
X = df[features].fillna(0.).values
y = df['SalePrice']
lasso.fit(X, np.log(y))

LassoLarsCV(copy_X=True, cv=None, eps=2.2204460492503131e-16,
      fit_intercept=True, max_iter=1000, max_n_alphas=1000, n_jobs=1,
      normalize=True, positive=False, precompute='auto', verbose=False)

In [227]:
yhat = np.exp(lasso.predict(X))
error(y,yhat)

0.11305713076847966

In [ ]:
yhat_test = lasso.predict(X_test)
yhat_test

In [225]:
#create csv file for kaggle submission
yhat_df = pd.DataFrame(yhat_test)
comp_df= pd.concat([test['Id'], yhat_df], axis =1)
comp_df=comp_df.rename(columns = {0:'SalePrice'})
comp_df.to_csv('comp_df', index =False)